# Assignment 3 - Data Science in Finance and Insurance
Submission by: Dennis Goldenberg

In [2]:
#import statements
import numpy as np;
import pandas as pd;
import plotnine as pn;

#set constants
A,B,c = 2.2e-4,2.7e-6,1.124

(A,B,c)

(0.00022, 2.7e-06, 1.124)

## Part [a] - Repeated Simpson's Rule for integration in Numpy

In [8]:
Makeham = lambda A,B,c,x,t : (A + B*(c**(x + t)))*np.exp(-1*A*t - (B/np.log(c))*(c**x)*((c**t)- 1))

Makeham(A,B,c,10,80)

0.04186595103341114